In [1]:
from google.colab import drive
drive.mount("/content/drive")



Mounted at /content/drive


Encode to 1D matrix

In [11]:
import pandas, csv ,torch
from pandas import Series
def encode(s): #translate
    a = []
    dic = {'A':1,'B':22,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,
           'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,
           'V':18,'W':19,'Y':20,'X':21,'U':23,'J':24,'Z':25,'O':26}
    for c in s:
      a.append(dic[c])
    return a

def list2Tensor(l:list):
  return torch.tensor(l)

import numpy as np
def list2np(l:list):
  a = np.array(l)
  return a

def getSeqFamily():
  return pandas.read_csv("/content/drive/My Drive/FYP/Data/all_Prot_Seq.csv")


{'Inoviridae': 0, 'Podoviridae': 1, 'Myoviridae': 2, 'Siphoviridae': 3}


In [21]:

#get seqs and families 
df = getSeqFamily()
#shuffle
df = df.sample(frac=1).reset_index(drop=True)
#One hot encoding
one_hot = pandas.get_dummies(df["family"])
#Drop column B as it is now encoded
df = df.drop("family",axis = 1)
#Join the encoded df
df = df.join(one_hot)
print(df)
"""
#Convert family name to 0 - x(3)
families = df["family"]
families_np = families.to_numpy()
families_set = set(families_np)
families_list = list(families_set)
families_dict = {}
for i in range(len(families_list)):
  families_dict[families_list[i]] = i
print(families_dict)
"""
SPLIT_RATIO = 0.8


Inoviridae
         prot_tag         prot_id  ... Podoviridae  Siphoviridae
0     VCO139_0062      AGI61889.1  ...           1             0
1         pp2_156      AFN37389.1  ...           0             0
2       VpV262p58     NP_640319.1  ...           1             0
3     AVU32_gp031  YP_009201134.1  ...           0             0
4      ASR73825.1      ASR73825.1  ...           0             0
...           ...             ...  ...         ...           ...
2880  AVU32_gp100  YP_009201203.1  ...           0             0
2881   VPMG_00074  YP_007676564.1  ...           1             0
2882   VPFG_00008  YP_008125165.2  ...           0             0
2883     VP5_gp02     YP_024977.1  ...           1             0
2884  phiST2_0165      ALP47353.1  ...           0             0

[2885 rows x 10 columns]


Config Torch dataset


In [32]:
SPLIT_RATIO = 0.8
END_INDEX = int(len(df.index)*SPLIT_RATIO)
training_x = df.iloc[0:END_INDEX]["seq"]
test_x = df.iloc[END_INDEX:len(df.index)]["seq"]

training_y = df.iloc[0:END_INDEX][one_hot.columns]
test_y = df.iloc[END_INDEX:len(df.index)][one_hot.columns]
print(test_y)


      Inoviridae  Myoviridae  Podoviridae  Siphoviridae
2308           0           1            0             0
2309           0           0            1             0
2310           0           1            0             0
2311           0           1            0             0
2312           0           1            0             0
...          ...         ...          ...           ...
2880           0           1            0             0
2881           0           0            1             0
2882           0           1            0             0
2883           0           0            1             0
2884           0           1            0             0

[577 rows x 4 columns]


To numpy

In [ ]:
from Bio import Entrez, SeqIO
from datetime import datetime
import time ,random
from multiprocessing import Process
from multiprocessing import Pool
import multiprocessing
import pandas, csv


def readCSV():
    for csv in CSVs:
        try:
            data = pandas.read_csv("/content/drive/My Drive/FYP/"+csv)
            data = data.dropna(how='all')
            print(data)
        except Exception as e:
            print(e)
            data = data.dropna(how='all')
            data = pandas.read_csv(csv)
            print(data)
        dataL.append(data)


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


        prot_tag         prot_id  ... Unnamed: 1023 Unnamed: 1024
0     VPKG_00030  YP_007673992.1  ...           NaN           NaN
1      AEO54_249      AKY02254.1  ...           NaN           NaN
2    AVV29_gp087  YP_009207589.1  ...           NaN           NaN
3      AEO54_253      AKY02255.1  ...           NaN           NaN
4    THALASSA_37      AUG85239.1  ...           NaN           NaN
..           ...             ...  ...           ...           ...
316   VPEG_00085  YP_004957577.1  ...           NaN           NaN
317   AOQ26687.1      AOQ26687.1  ...           NaN           NaN
318   AEH21887.1      AEH21887.1  ...           NaN           NaN
319   AEH21815.1      AEH21815.1  ...           NaN           NaN
320    SHOU24_91  YP_009006492.1  ...           NaN           NaN

[321 rows x 1025 columns]
05:00:40 Sipho_Prot_Seq.csv Doing index:0 YP_007673992.1
05:00:40 Sipho_Prot_Seq.csv Doing index:1 AKY02254.1
05:00:40 Sipho_Prot_Seq.csv Doing index:2 YP_009207589.1
05:00:40 Sipho